# Example: Calculating Underground Fluxes

This file demonstrates how to use MUTE to calculate underground fluxes for a lab located 3.7 km.w.e. under rock.

## Import Packages

In [1]:
import numpy as np

import mute.constants as mtc
import mute.surface as mts
import mute.propagation as mtp
import mute.underground as mtu

## Set the Constants

In [2]:
mtc.set_verbose(2)
mtc.set_output(True)
mtc.set_lab("Example")
mtc.set_overburden("flat")
mtc.set_vertical_depth(3.7)
mtc.set_medium("rock")
mtc.set_density(2.65)
mtc.set_n_muon(100000)

## Check the Slant Depths

Check the number and value of the slant depths that the underground fluxes will be calculated with. Because the vertical depth was set to ``3.7`` above, the slant depths should start at 3.7 km.w.e. The number of slant depths should be reduced from the default 28 to 22.

In [3]:
print(mtc.slant_depths)
print(len(mtc.slant_depths))

[ 3.7  4.   4.5  5.   5.5  6.   6.5  7.   7.5  8.   8.5  9.   9.5 10.
 10.5 11.  11.5 12.  12.5 13.  13.5 14. ]
22


The underground fluxes will be calculated for the zenith angles corresponding to these slant depths. The correspondence is given by this equation:

$$\theta=\arccos\left(\frac{h}{X}\right)=\arccos\left(\frac{3.7\ \mathrm{km.w.e.}}{X}\right)$$

## Calculate the Underground Fluxes

The ``mtu.calc_u_fluxes()`` function will return a matrix of shape ``(91, 22)`` for the 91 energies in the grid given by ``mtc.ENERGIES``, and the 22 angles corresponding to the slant depths above.

In [4]:
u_fluxes = mtu.calc_u_fluxes()

print(u_fluxes)
print(u_fluxes.shape)

Calculating underground fluxes.
Loading surface fluxes for USStd using GSF and SIBYLL-2.3c.
Loaded surface fluxes.
Loading survival probabilities from data/survival_probabilities/rock_2.65_100000_Survival_Probabilities.txt.
Loaded survival probabilities.
Finished calculating underground fluxes.
Underground fluxes written to data/underground/Example_Underground_Fluxes.txt.
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.67019874e-14 5.77016925e-14 2.88508321e-14 ... 3.45472040e-18
  3.05927169e-20 7.65680563e-21]
 [1.59920131e-13 1.43532627e-13 3.68028308e-14 ... 1.78156633e-18
  5.58709559e-18 2.29089741e-19]
 ...
 [3.63895510e-35 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(91, 22)


If the full tensor is needed, this can be obtained by setting ``full_tensor`` to ``True`` in the function call. This will return a three-dimensional array of shape ``(28, 91, 20)``, for the default slant depths, energies, and surface flux zenith angles given by ``mtc._SLANT_DEPTHS``, ``mtc.ENERGIES``, and ``mtc.ANGLES_FOR_S_FLUXES`` respectively. The output file will also contain the full tensor.

In [5]:
u_fluxes_full = mtu.calc_u_fluxes(full_tensor = True)

print(u_fluxes_full[0, :5, :5])
print(u_fluxes_full.shape)

Calculating underground fluxes.
Loading surface fluxes for USStd using GSF and SIBYLL-2.3c.
Loaded surface fluxes.
Survival probabilities already loaded for rock with density 2.65 gcm^-3 and 100000 muons.
Finished calculating underground fluxes.
Underground fluxes written to data/underground/Example_Underground_Fluxes.txt.
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [5.40109447e-11 5.40751625e-11 5.42782299e-11 5.46175089e-11
  5.50734247e-11]
 [1.34487970e-10 1.34649717e-10 1.35160145e-10 1.36013422e-10
  1.37161474e-10]
 [2.13325466e-10 2.13579177e-10 2.14381511e-10 2.15722044e-10
  2.17523509e-10]
 [2.25263220e-10 2.25532132e-10 2.26381881e-10 2.27801891e-10
  2.29710915e-10]]
(28, 91, 20)


## Calculating Fluxes from Pre-Calculated Matrices

If the code already has a surface flux matrix and / or survival probability tensor defined, they can be passed directly into the ``mtu.calc_u_fluxes()`` function. This is especially useful when dealing with multiple surface flux matrices and / or survival probability tensors in the same file, like when calculating various quantities by looping over variables. For example, to calculate both the surface fluxes and underground fluxes for different locations, the following can be done:

In [6]:
locations = ["SoudanMine", "SanGrasso", "Tokyo"]

all_s_fluxes = []
all_u_fluxes = []

sp_tensor = mtp.load_survival_probability_tensor_from_file()

for loc in locations:
    
    s_fluxes = mts.calc_s_fluxes(atmosphere = "MSIS00", location = loc, month = "January")
    u_fluxes = mtu.calc_u_fluxes(s_fluxes = s_fluxes, survival_probability_tensor = sp_tensor)
    
    all_s_fluxes.append(s_fluxes)
    all_u_fluxes.append(u_fluxes)

Survival probabilities already loaded for rock with density 2.65 gcm^-3 and 100000 muons.
Calculating surface fluxes for SoudanMine using GSF and SIBYLL-2.3c.
MCEqRun::set_interaction_model(): SIBYLL23C
MCEqRun::set_density_model(): Setting density profile to CORSIKA ('BK_USStd', None)
MCEqRun::set_primary_model(): GlobalSplineFitBeta 
MCEqRun::set_density_model(): Setting density profile to MSIS00 ('SoudanMine', 'January')


100%|██████████████████████████████████████████████| 20/20 [01:44<00:00,  5.24s/it]


Finished calculating surface fluxes.
Surface fluxes written to data/surface/Surface_Fluxes_SoudanMine_January_SIBYLL-2.3c_GSF.txt.
Calculating underground fluxes.
Finished calculating underground fluxes.
Underground fluxes written to data/underground/Example_Underground_Fluxes.txt.
Calculating surface fluxes for SanGrasso using GSF and SIBYLL-2.3c.
MCEqRun::set_interaction_model(): SIBYLL23C
MCEqRun::set_density_model(): Setting density profile to CORSIKA ('BK_USStd', None)
MCEqRun::set_primary_model(): GlobalSplineFitBeta 
MCEqRun::set_density_model(): Setting density profile to MSIS00 ('SanGrasso', 'January')


100%|██████████████████████████████████████████████| 20/20 [01:45<00:00,  5.29s/it]


Finished calculating surface fluxes.
Surface fluxes written to data/surface/Surface_Fluxes_SanGrasso_January_SIBYLL-2.3c_GSF.txt.
Calculating underground fluxes.
Finished calculating underground fluxes.
Underground fluxes written to data/underground/Example_Underground_Fluxes.txt.
Calculating surface fluxes for Tokyo using GSF and SIBYLL-2.3c.
MCEqRun::set_interaction_model(): SIBYLL23C
MCEqRun::set_density_model(): Setting density profile to CORSIKA ('BK_USStd', None)
MCEqRun::set_primary_model(): GlobalSplineFitBeta 
MCEqRun::set_density_model(): Setting density profile to MSIS00 ('Tokyo', 'January')


100%|██████████████████████████████████████████████| 20/20 [01:44<00:00,  5.24s/it]

Finished calculating surface fluxes.
Surface fluxes written to data/surface/Surface_Fluxes_Tokyo_January_SIBYLL-2.3c_GSF.txt.
Calculating underground fluxes.
Finished calculating underground fluxes.
Underground fluxes written to data/underground/Example_Underground_Fluxes.txt.


Like this, the loop will not have to be broken to calculate the underground fluxes, and the surface fluxes will not have to be calculated again inside the ``mtu.calc_u_fluxes()`` function like they are in the ``mts.calc_s_fluxes()`` function, because they are being passed into it directly.

The functions for surface intensities and total fluxes can take ``s_fluxes`` matrices in as well, and the functions for underground intensities and total fluxes can take in both ``s_fluxes`` and ``survival_probability_tensor`` arguments.